# Analysis of our data

1. Data collecting and data cleaning 
2. Computations for the creation of the adjacency matrix and the graph
3. Graph creation
4. Analysis

In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
from pandas.io.json import json_normalize
import pickle
import re
import scipy as sp
from scipy import stats
import seaborn as sns
from sklearn import metrics
from sklearn.cluster import DBSCAN
from sklearn.datasets.samples_generator import make_blobs
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import StandardScaler
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
pd.options.mode.chained_assignment = None
import datetime
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

import omdb

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zouag\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\zouag\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
%matplotlib inline

# 1. Data collecting and data cleaning

In [3]:
#import dataset
FILE_PATH_CREW = "../Data/tmdb_5000_credits.csv"

features_crew = pd.read_csv(FILE_PATH_CREW)

# Dataframe with info about the movies
features_crew.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [ ]:
#specify your API_KEY used to set client
API_y_KEY = '3a7e30ce'
omdb.set_default('apikey', API_y_KEY)
client = omdb.OMDBClient(apikey=API_y_KEY)

In [ ]:
#We have a 1000 request limit so you can decide here at which film you want to start and where to end 
id_end = 4803
id_start = id_end - 1000

In [ ]:
#creates empty dictionary then updates it using the movie_id as a key and the output of the
#request as value (note that that res.content isn't actually a dict but a byte object, we may
#need to convert it later)

collected = dict()

for i in range(id_start, id_end):
    title = features_crew.title[i]
    res = omdb.request(t=title, tomatoes=True)
    movie_json = res.content
    collected[str(features_crew.movie_id[i])] = movie_json

In [ ]:
# saves collected as pickle
with open('metacrit.p', 'wb') as fp:
    pickle.dump(collected, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# loads collected
with open('metacrit.p', 'rb') as fp:
    test_data = pickle.load(fp)

# 2. Computations for the creation of the adjacency matrix and the graph

In [ ]:
df = pd.DataFrame.from_dict(test_data, orient='index')
df.head()

In [ ]:
dataframe = pd.read_json(df.iloc[0].values[0])

for i in range(1, len(df)):
    if len(df.iloc[i].values[0]) > 100:
        movie = pd.read_json(df.iloc[i].values[0])
    dataframe = pd.concat([dataframe, movie])

In [ ]:
# We decided to drop the useless columns that did not contain information about the movie
# These columns are: - urls (Poster, Website, tomatoImage)
#                    - columns related to the API (Response, Type)
#                    - columns where all entries are NaN
#                    - Irrelevant columns (DVD, ie release date of dvd)
#                    - Redundant columns (Year is redundant with Released)

dataframe = dataframe.drop(columns=['Poster', 'Response', 'Type', 'Website', 'tomatoConsensus', 
                                    'tomatoFresh', 'tomatoImage', 'tomatoMeter', 'tomatoRating', 
                                    'tomatoReviews', 'tomatoRotten', 'tomatoURL', 'tomatoUserMeter', 
                                    'tomatoUserRating', 'tomatoUserReviews', 'totalSeasons', 'DVD', 'Year'])
dataframe.head()

We now need to transform the types so that it is easier to analyze the features. As of now, almost all the entries of our dataframe are strings. We have a few entries that are integer numbers that need to be converted: BoxOffice, Metascore, Runtime and imdbVotes.

Metascore is already in an appropriate format, we just need to cast the type. However, BoxOffice, Runtime and imdbVotes need a little bit of tinkering.

In [ ]:
# For BoxOffice we need to remove the $ and commas
dataframe['BoxOffice'] = dataframe['BoxOffice'].apply(lambda x: x.replace("$", ""))
dataframe['BoxOffice'] = dataframe['BoxOffice'].apply(lambda x: x.replace(",", ""))
dataframe['BoxOffice'] = dataframe['BoxOffice'].replace('N/A', 0)
dataframe['BoxOffice'] = dataframe['BoxOffice'].astype(float)
dataframe['BoxOffice'] = dataframe['BoxOffice'].astype(int)
# For imdbVotes we need to remove the commas
dataframe['imdbVotes'] = dataframe['imdbVotes'].astype(str)
dataframe['imdbVotes'] = dataframe['imdbVotes'].apply(lambda x: x.replace(",", ""))
dataframe['imdbVotes'] = dataframe['imdbVotes'].replace('N/A', 0)
dataframe['imdbVotes'] = dataframe['imdbVotes'].astype(int)
# For Runtime we need to remove the " min"
dataframe['Runtime'] = dataframe['Runtime'].apply(lambda x: x.replace(" min", ""))
dataframe['Runtime'] = dataframe['Runtime'].replace('N/A', 0)
dataframe['Runtime'] = dataframe['Runtime'].astype(int)
# Metascore
dataframe['Metascore'] = dataframe['Metascore'].replace('N/A', 0)
dataframe['Metascore'] = dataframe['Metascore'].astype(int)

dataframe = dataframe.reset_index(drop=True)

In [ ]:
dataframe.head()

Now, we need to regroup the different sources of ratings so that the movies are not repeated three times in the dataframe.

- The first rating comes from imdb, its value is on a scale from 0 to 10
- The second rating comes from Rotten Tomatoes, its value is a percentage
- The third rating comes from Metacritic, its value is on a scale from 0 to 100

We wish to separate this column into 3 columns, drop the duplicate rows and convert the ratings to percentages for a more uniform notation.

In [ ]:
dataframe_ = dataframe.copy()

In [ ]:
# Create columns
dataframe_['imdbGrade'] = 0
dataframe_['tomatoGrade'] = 0
dataframe_['metacriticGrade'] = 0

# Fill columns
for i in range(0, int(len(dataframe)/3)):
    dataframe_['tomatoGrade'][3*i] = dataframe_['Ratings'][3*i+1]['Value']

# Drop rows
dataframe_ = dataframe_[dataframe_['tomatoGrade'] != 0]

In [ ]:
dataframe_ = dataframe_.reset_index(drop=True)

In [ ]:
# Transform grades
dataframe_['imdbGrade'] = dataframe_['imdbRating']*10
dataframe_ = dataframe_.drop(columns=['imdbRating'])

for i in range(0, len(dataframe_)):
    if '/10' in dataframe_['tomatoGrade'][i]:
        grade_ = dataframe_['tomatoGrade'][i].replace('/10', '').replace('.', '')
        dataframe_['tomatoGrade'][i] = int(grade)
    elif '%' in dataframe_['tomatoGrade'][i]:
        grade = dataframe_['tomatoGrade'][i].replace('%', '')
        dataframe_['tomatoGrade'][i] = int(grade)
    elif '/100' in dataframe_['metacriticGrade'][i]:
        grade = dataframe_['metacriticGrade'][i].replace('/100', '')
        dataframe_['metacriticGrade'][i] = int(grade)
    else:
        grade = float(dataframe_['tomatoGrade'][i])
        if grade > 100:
            grade = grade / 10
        dataframe_['tomatoGrade'][i] = int(grade)

dataframe_['metacriticGrade'] = dataframe_['Metascore']
dataframe_ = dataframe_.drop(columns=['Metascore'])

In [ ]:
dataframe_['imdbGrade'] = dataframe_['imdbGrade'].astype(int)
dataframe_['tomatoGrade'] = dataframe_['tomatoGrade'].astype(int)
dataframe_['metacriticGrade'] = dataframe_['metacriticGrade'].astype(int)
dataframe_ = dataframe_.drop(columns=['Ratings'])

In [118]:
dataframe_.head()

,Actors,BoxOffice,Country,Director,Genre,Language,Plot,Production,Rated,Released,Runtime,Title,Writer,imdbID,imdbVotes,imdbGrade,tomatoGrade,metacriticGrade,Wins,Nominations
0,"{Stephen Lang, Sigourney Weaver, Zoe Saldana, ...",749700000,"{Foreign, USA}",{James Cameron},"{Adventure, Fantasy, Sci-Fi, Action}","{Spanish, English}","{home, following, paraplegic, unique, missionc...",{20th Century Fox},{PG-13},2009-12-18,162,Avatar,{James Cameron},tt0499549,1013715,78,82,83,88,128
1,"{Keira Knightley, Geoffrey Rush, Orlando Bloom...",309404152,{USA},{Gore Verbinski},"{Adventure, Fantasy, Action}",{English},"{Turnernd, Barbossa, last, treacheryndtrayal, ...",{Buena Vista},{PG-13},2007-05-25,169,Pirates of the Caribbean: At World's End,"{Terry Rossio, Jay Wolpert, Ted Elliott, Stuar...",tt0449088,542125,71,44,50,20,46
2,"{Daniel Craig, Léa Seydoux, Ralph Fiennes, Chr...",208777731,"{Foreign, USA}",{Sam Mendes},"{Adventure, Action, Thriller}","{English, Spanish, Other}","{007, events, missions, sends, called, terrori...",{Sony},{PG-13},2015-11-06,148,Spectre,"{John Logan, Jez Butterworth, Robert Wade, Nea...",tt2379713,343533,68,64,60,8,32
3,"{Gary Oldman, Joseph Gordon-Levitt, Christian ...",448130642,"{Foreign, USA}",{Christopher Nolan},"{Action, Thriller}","{English, Foreign}","{Joker, terrorist, Gotham, City, exile, Eight,...",{Warner Bros},{PG-13},2012-07-20,164,The Dark Knight Rises,"{Christopher Nolan, David S. Goyer, Jonathan N...",tt1345836,1344474,84,87,78,38,103
4,"{Samantha Morton, Willem Dafoe, Taylor Kitsch,...",73058679,{USA},{Andrew Stanton},"{Adventure, Sci-Fi, Action}",{English},"{vet, princess, War, creatures, barbarians, Tr...",{Disney},{PG-13},2012-03-09,132,John Carter,"{Edgar Rice Burroughs, Andrew Stanton, Michael...",tt0401729,234717,66,51,51,2,8


Now, we need to deal with the string attributes that are in form of a list. We want a set of strings, each string corresponding to an individual characteristic (country, actor, etc).

In order to do so, we split the string at the commas. For the Writer attribute, we decided to remove the specifications and only keep the names of the writers, taking off who wrote the story, screenplay or characters.

In [ ]:
dataframe_['Actors'] = dataframe_['Actors'].apply(lambda x: set(x.split(', ')))
dataframe_['Country'] = dataframe_['Country'].apply(lambda x: set(x.split(', ')))
dataframe_['Director'] = dataframe_['Director'].apply(lambda x: set(x.split(', ')))
dataframe_['Genre'] = dataframe_['Genre'].apply(lambda x: set(x.split(', ')))
dataframe_['Language'] = dataframe_['Language'].apply(lambda x: set(x.split(', ')))

In [ ]:
dataframe_['Writer'] = dataframe_['Writer'].apply(lambda x: re.sub(" [\(\[].*?[\)\]]", "", x))
dataframe_['Writer'] = dataframe_['Writer'].apply(lambda x: set(x.split(', ')))

In [4]:
oscars_df = pd.read_csv('../Data/oscars.csv')
# Actor & actress (supporting or Leading role)
# Directing, Directing (Comedy Picture), Directing (Dramatic Picture)
# All the Writings

set(oscars_df.Award)
oscars_df.Year = oscars_df.Year.apply(lambda x: int(x[0:4]) if len(x) > 4 else int(x))
oscars_df= oscars_df[oscars_df.Year.apply(lambda x: x > 1930)].dropna()

In [5]:
oscars_df.head()

,Year,Ceremony,Award,Winner,Name,Film
159,1931,5,Actor,1.0,Wallace Beery,The Champ
161,1931,5,Actor,1.0,Fredric March,Dr. Jekyll and Mr. Hyde
164,1931,5,Actress,1.0,Helen Hayes,The Sin of Madelon Claudet
167,1931,5,Art Direction,1.0,Transatlantic,Gordon Wiles
170,1931,5,Cinematography,1.0,Shanghai Express,Lee Garmes


# comment on what am doing here

In [17]:
nominated_dir = oscars_df[oscars_df.Award.apply(lambda x: 'Directing' in x)][['Film']]
nominated_dir = nominated_dir.rename(columns={'Film' : 'Name'})

nominated_writ = oscars_df[oscars_df.Award.apply(lambda x: 'Writing' in x)][['Film']]
nominated_writ = nominated_writ.rename(columns={'Film' : 'Name'})

nominated_actors = oscars_df[oscars_df.Award.apply(lambda x: 'Actor' in x or 'Actress' in x)][['Name']]

awards = pd.concat([nominated_dir, nominated_writ, nominated_actors], axis = 0)

In [ ]:
nominated_writ.Name = nominated_writ.Name.apply(lambda x: x.replace('Written by ', ''))
nominated_writ.Name = nominated_writ.Name.apply(lambda x: x.replace('Screenplay by ', ''))
nominated_writ.Name = nominated_writ.Name.apply(lambda x: x.replace('Written for the screen by ', ''))
nominated_writ.Name = nominated_writ.Name.apply(lambda x: x.replace('Story by', ''))
nominated_writ.Name = nominated_writ.Name.apply(lambda x: x.replace('(', ''))
nominated_writ.Name = nominated_writ.Name.apply(lambda x: x.replace(')', ''))
nominated_writ.Name = nominated_writ.Name.apply(lambda x: x.replace(' &', ','))
nominated_writ.Name = nominated_writ.Name.apply(lambda x: x.replace(';', ','))
nominated_writ.Name = nominated_writ.Name.apply(lambda x: x.replace(' and ', ', '))

In [18]:
awards.Name = awards.Name.apply(lambda x: x.replace('Written by ', ''))
awards.Name = awards.Name.apply(lambda x: x.replace('Screenplay by ', ''))
awards.Name = awards.Name.apply(lambda x: x.replace('Written for the screen by ', ''))
awards.Name = awards.Name.apply(lambda x: x.replace('Adaptation by ', ''))
awards.Name = awards.Name.apply(lambda x: x.replace('Dialogue by ', ''))
awards.Name = awards.Name.apply(lambda x: x.replace('Story by ', ''))
awards.Name = awards.Name.apply(lambda x: x.replace('Stories by ', ''))
awards.Name = awards.Name.apply(lambda x: x.replace('(', ''))
awards.Name = awards.Name.apply(lambda x: x.replace(')', ''))
awards.Name = awards.Name.apply(lambda x: x.replace(' &', ','))
awards.Name = awards.Name.apply(lambda x: x.replace(';', ','))
awards.Name = awards.Name.apply(lambda x: x.replace(' and ', ', '))
awards.Name = awards.Name.str.replace(', Jr.', ' Jr.')
awards.Name = awards.Name.str.replace('-', ' ')
awards.Name = awards.Name.str.strip()

In [117]:
award = pd.DataFrame(awards.Name.str.split(', ').tolist()).stack()
award = award.reset_index()[[0]]
award.columns = ['Names']

award = award.groupby('Names').agg({"Names": 'count'})
award.rename(columns={'Names': 'Awards'}, inplace=True)
award.reset_index(inplace=True)

award.head()

,Names,Awards
0,Aaron Sorkin,1
1,Abby Mann,1
2,Adam McKay,1
3,Adrien Brody,1
4,Akiva Goldsman,1


In [184]:
crew_act = dataframe_.loc[:,['Actors', 'Title']]
crew_dir = dataframe_.loc[:,['Director', 'Title']]
crew_writ = dataframe_.loc[:,['Writer', 'Title']]

crew_act.Actors = crew_act.Actors.apply(lambda x: ', '.join(list(x)))
crew_dir.Director = crew_dir.Director.apply(lambda x: ', '.join(list(x)))
crew_writ.Writer = crew_writ.Writer.apply(lambda x: ', '.join(list(x)))

crew_act = pd.DataFrame(crew_act.Actors.str.split(', ').tolist(), index=crew_act.Title).stack()
crew_act = crew_act.reset_index()[[0, 'Title']]
crew_act.columns = ['Actor', 'Title']

crew_dir = pd.DataFrame(crew_dir.Director.str.split(', ').tolist(), index=crew_dir.Title).stack()
crew_dir = crew_dir.reset_index()[[0, 'Title']]
crew_dir.columns = ['Director', 'Title']

crew_writ = pd.DataFrame(crew_writ.Writer.str.split(', ').tolist(), index=crew_writ.Title).stack()
crew_writ = crew_writ.reset_index()[[0, 'Title']]
crew_writ.columns = ['Writer', 'Title']

crew_writ.head()

,Writer,Title
0,James Cameron,Avatar
1,Terry Rossio,Pirates of the Caribbean: At World's End
2,Jay Wolpert,Pirates of the Caribbean: At World's End
3,Ted Elliott,Pirates of the Caribbean: At World's End
4,Stuart Beattie,Pirates of the Caribbean: At World's End


In [185]:
dataframe_tmp = dataframe_.loc[:,['Title', 'BoxOffice', 'Actors', 'Director', 'Writer']]
dataframe_tmp.columns = ['Title', 'BoxOffice', 'Actors', 'Directors', 'Writers']

crew_act = pd.merge(crew_act, dataframe_tmp, on='Title')[['Actor', 'BoxOffice']]
crew_act = crew_act.groupby('Actor').agg({"BoxOffice": 'sum'})
crew_act.rename(columns={'Actor': 'BoxOffice'}, inplace=True)
crew_act.reset_index(inplace=True)

crew_dir = pd.merge(crew_dir, dataframe_tmp, on='Title')[['Director', 'BoxOffice']]
crew_dir = crew_dir.groupby('Director').agg({"BoxOffice": 'sum'})
crew_dir.rename(columns={'Director': 'BoxOffice'}, inplace=True)
crew_dir.reset_index(inplace=True)

crew_writ = pd.merge(crew_writ, dataframe_tmp, on='Title')[['Writer', 'BoxOffice']]
crew_writ = crew_writ.groupby('Writer').agg({"BoxOffice": 'sum'})
crew_writ.rename(columns={'Writer': 'BoxOffice'}, inplace=True)
crew_writ.reset_index(inplace=True)

crew_writ.head()

,Writer,BoxOffice
0,'Weird Al' Yankovic,0
1,A. Raven Cruz,0
2,A.A. Milne,44701110
3,A.B. 'Banjo' Paterson,0
4,A.E.W. Mason,18236897


Now, we deal with the Production company. We have observed that a same company has different names within this dataset, sometimes the names diferring only by a space or an add-on (like "distribution"). We want to uniformize this.

In [ ]:
dataframe_['Production'] = dataframe_['Production'].str.replace('-', ' ')
dataframe_['Production'] = dataframe_['Production'].str.replace(' / ', '/')
dataframe_['Production'] = dataframe_['Production'].str.replace(' /', '/')
dataframe_['Production'] = dataframe_['Production'].str.replace('/ ', '/')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Corporation', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Distribution', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Corporat', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Production', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Films', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Film', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Pictures', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Picutres', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Internationa', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' International', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Industries', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Compa', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Co', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Distribu', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Studios', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Animation', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Feature', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' SKG', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' LLC', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Recommends', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Digital', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Media', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Video', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Inc', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Home Entertainment', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Entertainment', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Lorber', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Releasing', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Cinema.', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Cinema', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Classics', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Classic', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Group', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Europacorp', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Pvt. Ltd.', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Ventures', '')
dataframe_['Production'] = dataframe_['Production'].str.replace('DreamWorks', 'Dreamworks')
dataframe_['Production'] = dataframe_['Production'].str.replace('Dream Works', 'Dreamworks')
dataframe_['Production'] = dataframe_['Production'].str.replace("Lion's Gate", 'Lionsgate')
dataframe_['Production'] = dataframe_['Production'].str.replace("Liongate", 'Lionsgate')
dataframe_['Production'] = dataframe_['Production'].str.replace("Lions Gate", 'Lionsgate')
dataframe_['Production'] = dataframe_['Production'].str.replace("LionsGate", 'Lionsgate')
dataframe_['Production'] = dataframe_['Production'].str.replace('Metro Goldwyn Mayer (MGM)', 'MGM')
dataframe_['Production'] = dataframe_['Production'].str.replace('Metro Goldwyn Mayer', 'MGM')
dataframe_['Production'] = dataframe_['Production'].str.replace('WARNER BROTHERS PICTURES', 'Warner Bros')
dataframe_['Production'] = dataframe_['Production'].str.replace('WB', 'Warner Bros')
dataframe_['Production'] = dataframe_['Production'].str.replace('Weinsteinny', 'Weinstein')
dataframe_['Production'] = dataframe_['Production'].str.replace('Warner Brothers', 'Warner Bros')
dataframe_['Production'] = dataframe_['Production'].str.replace('Warner Home', 'Warner Bros')
dataframe_['Production'] = dataframe_['Production'].str.replace('Warner Independent', 'Warner Bros')
dataframe_['Production'] = dataframe_['Production'].str.replace('PIXAR', 'Pixar')
dataframe_['Production'] = dataframe_['Production'].str.replace(' [us]', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Pictur', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Home', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Focus', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' City', '')
dataframe_['Production'] = dataframe_['Production'].str.replace('Warners Bros', 'Warner Bros')
dataframe_['Production'] = dataframe_['Production'].str.replace('Universall', 'Universal')
dataframe_['Production'] = dataframe_['Production'].str.replace('The Weinstein', 'Weinstein')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Vantage', 'Weinstein')
dataframe_['Production'] = dataframe_['Production'].str.replace('/lumbia', '/ Columbia')
dataframe_['Production'] = dataframe_['Production'].str.replace('MGM (MGM)', 'MGM')
dataframe_['Production'] = dataframe_['Production'].str.replace(' First Take', '/ Columbia')
dataframe_['Production'] = dataframe_['Production'].str.replace('Goodbye Cruel', 'Goodbye Cruel World')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Atomic', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Faith', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Searchlight', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Walden', '')
dataframe_['Production'] = dataframe_['Production'].str.replace('FoxFaith', 'Fox')
dataframe_['Production'] = dataframe_['Production'].str.replace('Fine Line', 'Fine Lines')
dataframe_['Production'] = dataframe_['Production'].str.replace('Erosl', 'Eros')
dataframe_['Production'] = dataframe_['Production'].str.replace('Walt Disneys', 'Disney')
dataframe_['Production'] = dataframe_['Production'].str.replace('Walt Disney', 'Disney')
dataframe_['Production'] = dataframe_['Production'].str.replace('.', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' TriStarl', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' TriStar', '')
dataframe_['Production'] = dataframe_['Production'].str.replace(' Tristar', '')

In [ ]:
dataframe_['Production'] = dataframe_['Production'].apply(lambda x: set(x.split('/')))

Now, we deal with the plot. For this NLP task, we will keep it simple. We only kept the common nouns (singular or plural).

In [ ]:
def find_nouns(text):
    tokens = nltk.word_tokenize(text)
    tags = nltk.pos_tag(tokens)
    nouns = [word for word,pos in tags if (pos == 'NN' or pos == 'NNS')]
    return set(nouns)

In [ ]:
dataframe_['Plot'] = dataframe_['Plot'].apply(lambda x: find_nouns(x))

We have found that among all movies, there are a lot of distinct words. For our machine learning analysis, this can be problematic because each word is one feature. 

In [ ]:
total_set = set()
for i in range(len(dataframe_)):
    total_set = total_set.union(dataframe_['Plot'][i])
    
len(total_set)

Indeed, there are 5734 words. We want to look at the distribution of the words.

In [ ]:
total_list = []
for i in range(len(dataframe_)):
    total_list = total_list + list(dataframe_['Plot'][i])

In [ ]:
from collections import Counter
word_counts = Counter(total_list)
word_df = pd.DataFrame.from_dict(word_counts, orient='index')

In [ ]:
df.plot(kind='hist', bins=100, figsize=(15, 10), logy=True, 
        title='Logarithm of histogram of the frequency of each plot word among all movies')
plt.show()

In [ ]:
df[df[0].apply(lambda x: x >= 100)]

The most common words among all the movies are the ones listed above. As we can see, most words do not describe the story (man, woman, life, school) but the scenario and the characters. We decided that the plot is useful for the graph creation but not for the machine learning analysis. 

We will use the plot for the graph creation (two similar movies will be movies that have a lot of plot words in common).

We need to format the Awards feature. The current Awards column contains a string (sentence) describing how many awards the movie has won or has been nominated to. It also specifies if it has won or been nominated to Oscars or BAFTA. We believe that when it is specified, the higher "award" is listed, i.e. if the movie won 2 Oscars, the 5 Oscar nominations won't be specified. The most useful format we could think of is to transform this award column into 2 distinct columns, since we cannot infer how many nominations to the Oscars a movie has when it has already won some. The columns are hence: 

- Wins
- Nominations

In [ ]:
def find_wins_nominations(string):
    wins = 0
    noms = 0
    
    splitted = string.split('. ')
    if len(splitted) > 1:
        string1 = splitted[0]
        if re.search('nominated', string1, re.IGNORECASE):
            noms = noms + int(string1.split(' ')[2])
        if re.search('won', string1, re.IGNORECASE):
            wins = noms + int(string1.split(' ')[1])
        
        string2 = splitted[1]
        if re.search('nomination', string2, re.IGNORECASE):
            noms = noms + int(string2.split(' nomination')[0].split(' ')[-1])
        if re.search('win', string2, re.IGNORECASE):
            wins = wins + int(string2.split(' win')[0].split(' ')[-1])
    elif len(splitted) == 1:
        split_string = splitted[0]
        if re.search('nomination', split_string, re.IGNORECASE):
            noms = noms + int(split_string.split(' nomination')[0].split(' ')[-1])
        if re.search('win', split_string, re.IGNORECASE):
            wins = wins + int(split_string.split(' win')[0].split(' ')[-1])
    return wins, noms

In [ ]:
dataframe_['Wins'] = 0
dataframe_['Nominations'] = 0

for i in range(0, len(dataframe_)):
    wins, noms = find_wins_nominations(dataframe_.Awards[i])
    dataframe_['Wins'][i] = wins
    dataframe_['Nominations'][i] = noms
dataframe_ = dataframe_.drop(columns='Awards')

The Rated attribute is not uniform and has multiple conventions. We seek to normalize it. We decided to use the Motion Picture Association of America film rating system, which states that:

- **G: General Audiences. All ages admitted.** Nothing that would offend parents for viewing by children.
- **PG: Parental Guidance Suggested.** Some material may not be suitable for children. Parents urged to give "parental guidance". May contain some material parents might not like for their young children.
- **PG-13: Parents Strongly Cautioned.** Some material may be inappropriate for children under 13. Parents are urged to be cautious. Some material may be inappropriate for pre-teenagers.
- **R: Restricted. Under 17 requires accompanying parent or adult guardian.** Contains some adult material. Parents are urged to learn more about the film before taking their young children with them.
- **NC-17: Adults Only.** No One 17 and Under Admitted. Clearly adult. Children are not admitted.

https://en.wikipedia.org/wiki/Motion_Picture_Association_of_America_film_rating_system

**X** rating means persons under 16 not admitted. For our analysis, we decided to conflate that label within **NC-17**

Following this, we need to make some modifications: 

- Approved -> G
- GP -> PG
- M -> R
- Passed -> G
- TV-14 -> PG-13
- TV-G -> G
- TV-MA -> R
- TV-PG -> PG
- X -> NC-17
- Unrated, Not rated and N/A make up their own category **Unrated**

In [ ]:
dataframe_['Rated'] = dataframe_['Rated'].str.replace('APPROVED', 'G')
dataframe_['Rated'] = dataframe_['Rated'].str.replace('GP', 'PG')
dataframe_['Rated'] = dataframe_['Rated'].str.replace('M', 'R')
dataframe_['Rated'] = dataframe_['Rated'].str.replace('PASSED', 'G')
dataframe_['Rated'] = dataframe_['Rated'].str.replace('TV-14', 'G')
dataframe_['Rated'] = dataframe_['Rated'].str.replace('APPROVED', 'PG-13')
dataframe_['Rated'] = dataframe_['Rated'].str.replace('TV-G', 'G')
dataframe_['Rated'] = dataframe_['Rated'].str.replace('TV-MA', 'R')
dataframe_['Rated'] = dataframe_['Rated'].str.replace('TV-PG', 'PG')
dataframe_['Rated'] = dataframe_['Rated'].str.replace('X', 'NC-17')
dataframe_['Rated'] = dataframe_['Rated'].str.replace('NOT RATED', 'UNRATED')
dataframe_['Rated'] = dataframe_['Rated'].str.replace('N/A', 'UNRATED')

In [ ]:
dataframe_['Rated'] = dataframe_['Rated'].apply(lambda x: {x})

In order to make it easier to analyze, we decided to transform every foreign language into the label "Foreign" except from Spanish.

In [ ]:
def format_languages(lang_set):
    if (lang_set - {'English', 'Spanish'}) != set(): # There are other things than English 
        if 'English' not in lang_set: # There is only other things than English
            if 'Spanish' not in lang_set:
                new_label = {'Other'}
            else: 
                new_label = {'Spanish', 'Other'}
        else:
            if 'Spanish' not in lang_set:
                new_label = {'English', 'Foreign'}
            else:
                new_label = {'English', 'Spanish', 'Other'}
    else:
        if 'English' not in lang_set:
            if 'Spanish' not in lang_set:
                new_label = set()
            else: 
                new_label = {'Spanish'}
        else:
            if 'Spanish' not in lang_set:
                new_label = {'English'}
            else:
                new_label = {'English', 'Spanish'}
    return new_label

In [ ]:
dataframe_['Language'] = dataframe_['Language'].apply(lambda x: format_languages(x))

The Release date is still a string. In order to handle it and make computations with it, we need to transform its format.

In [ ]:
def format_date(datestring):
    if datestring != 'N/A':
        splitted = datestring.split(' ')
        day = splitted[0]
        month = splitted[1]
        year = splitted[2]

        new_month = ''
        if month == 'Jan':
            new_month = '01'
        elif month == 'Feb':
            new_month = '02'
        elif month == 'Mar':
            new_month = '03'
        elif month == 'Apr':
            new_month = '04'
        elif month == 'May':
            new_month = '05'
        elif month == 'Jun':
            new_month = '06'
        elif month == 'Jul':
            new_month = '07'
        elif month == 'Aug':
            new_month = '08'
        elif month == 'Sep':
            new_month = '09'
        elif month == 'Oct':
            new_month = '10'
        elif month == 'Nov':
            new_month = '11'
        elif month == 'Dec':
            new_month = '12'

        return day+'/'+new_month+'/'+year
    else:
        return np.nan

In [ ]:
dataframe_['Released'] = dataframe_['Released'].apply(lambda x: format_date(x))
dataframe_['Released'] = pd.to_datetime(dataframe_['Released'], format='%d/%m/%Y')

The countries are formatted so as to show 'USA' or 'Foreign'.

In [ ]:
def format_country(country_set):
    if (country_set - {'USA'}) != set():
        if 'USA' not in country_set:
            new_label = {'Foreign'}
        else:
            new_label = {'USA', 'Foreign'}
    else:
        if 'USA' not in country_set:
            new_label = set()
        else:
            new_label = {'USA'}
    return new_label

In [ ]:
dataframe_['Country'] = dataframe_['Country'].apply(lambda x: format_country(x))

In [ ]:
dataframe_.to_pickle("df.pkl")

In [8]:
dataframe_ = pd.read_pickle("df.pkl")

Finally, we need to add the budget. For that, we use the budget column in the Kaggle imdb dataset.

In [ ]:
FILE_PATH_MOVIE = "../Data/tmdb_5000_movies.csv"
movie_df = pd.read_csv(FILE_PATH_MOVIE)

budget_df = movie_df[['budget', 'title']]
budget_df = budget_df.rename(columns={"budget" : "Budget", "title" : "Title"})

final_df = dataframe_.merge(budget_df, how='inner')

In [ ]:
final_df = final_df.drop(columns=['imdbID'])

In [ ]:
final_df.to_pickle("final_df.pkl")

In [197]:
dataframe_tmp = final_df.loc[:,['Actors','Director','Writer']]

In [198]:
dataframe_tmp

,Actors,Director,Writer
0,"{Stephen Lang, Sam Worthington, Sigourney Weav...",{James Cameron},{James Cameron}
1,"{Johnny Depp, Keira Knightley, Geoffrey Rush, ...",{Gore Verbinski},"{Terry Rossio, Jay Wolpert, Ted Elliott, Stuar..."
2,"{Ralph Fiennes, Christoph Waltz, Léa Seydoux, ...",{Sam Mendes},"{John Logan, Jez Butterworth, Robert Wade, Nea..."
3,"{Gary Oldman, Joseph Gordon-Levitt, Christian ...",{Christopher Nolan},"{Christopher Nolan, David S. Goyer, Jonathan N..."
4,"{Samantha Morton, Willem Dafoe, Taylor Kitsch,...",{Andrew Stanton},"{Edgar Rice Burroughs, Andrew Stanton, Michael..."
5,"{Kirsten Dunst, Thomas Haden Church, James Fra...",{Sam Raimi},"{Steve Ditko, Alvin Sargent, Sam Raimi, Ivan R..."
6,"{Ron Perlman, Donna Murphy, Zachary Levi, Mand...","{Nathan Greno, Byron Howard}","{Jacob Grimm, Dan Fogelman, Wilhelm Grimm}"
7,"{Chris Evans, Mark Ruffalo, Chris Hemsworth, R...",{Joss Whedon},"{Joe Simon, Joss Whedon, Jack Kirby, Jim Starl..."
8,"{Dave Legeno, Daniel Radcliffe, Michael Gambon...",{David Yates},"{J.K. Rowling, Steve Kloves}"
9,"{Jesse Eisenberg, Amy Adams, Henry Cavill, Ben...",{Zack Snyder},"{Chris Terrio, William Moulton Marston, Jerry ..."


In [189]:
crew_awards = pd.DataFrame(set(), index=np.arange(len(dataframe_tmp)), columns=['Names'])

for i in range(len(dataframe_tmp)):
    loc_list = list(dataframe_tmp.iloc[i])
    crew_awards.iloc[i].Names = loc_list[0]|loc_list[1]|loc_list[2]
crew_awards['Awards'] = 0

In [203]:
rev_act = pd.DataFrame(set(), index=np.arange(len(dataframe_tmp)), columns=['Actors'])
rev_act['Actors'] = dataframe_tmp.Actors
rev_act['BoxOffice_actors'] = 0

rev_dir = pd.DataFrame(set(), index=np.arange(len(dataframe_tmp)), columns=['Directors'])
rev_dir['Directors'] = dataframe_tmp.Director
rev_dir['BoxOffice_directors'] = 0

rev_writ = pd.DataFrame(set(), index=np.arange(len(dataframe_tmp)), columns=['Writers'])
rev_writ['Writers'] = dataframe_tmp.Writer
rev_writ['BoxOffice_writers'] = 0

In [212]:
for (s,set_) in enumerate(crew_awards.Names):
    for item_ in set_:
        if not award[award['Names']==item_].Awards.empty:
            crew_awards.at[s, 'Awards'] = crew_awards.at[s, 'Awards'] + award[award['Names']==item_].Awards      
        

In [205]:
for (s,set_) in enumerate(rev_act.Actors):
    for item_ in set_:
        rev_act.at[s, 'BoxOffice_actors'] = rev_act.at[s, 'BoxOffice_actors'] + crew_act[crew_act['Actor']==item_].BoxOffice      
        
for (s,set_) in enumerate(rev_dir.Directors):
    for item_ in set_:
        rev_dir.at[s, 'BoxOffice_directors'] = rev_dir.at[s, 'BoxOffice_directors'] + crew_dir[crew_dir['Director']==item_].BoxOffice      

for (s,set_) in enumerate(rev_writ.Writers):
    for item_ in set_:
        rev_writ.at[s, 'BoxOffice_writers'] = rev_writ.at[s, 'BoxOffice_writers'] + crew_writ[crew_writ['Writer']==item_].BoxOffice      
  

In [209]:
rev_writ

,Writers,BoxOffice_writers
0,{James Cameron},1173684544
1,"{Terry Rossio, Jay Wolpert, Ted Elliott, Stuar...",6439780219
2,"{John Logan, Jez Butterworth, Robert Wade, Nea...",4338225868
3,"{Christopher Nolan, David S. Goyer, Jonathan N...",6820715653
4,"{Edgar Rice Burroughs, Andrew Stanton, Michael...",2082778011
5,"{Steve Ditko, Alvin Sargent, Sam Raimi, Ivan R...",7500633793
6,"{Jacob Grimm, Dan Fogelman, Wilhelm Grimm}",1328044695
7,"{Joe Simon, Joss Whedon, Jack Kirby, Jim Starl...",10611250708
8,"{J.K. Rowling, Steve Kloves}",2760229621
9,"{Chris Terrio, William Moulton Marston, Jerry ...",6185995767


In [213]:
final_df['Awards'] = crew_awards.Awards
final_df['BoxOffice_actors'] = rev_act.BoxOffice_actors
final_df['BoxOffice_directors'] = rev_dir.BoxOffice_directors
final_df['BoxOffice_writers'] = rev_writ.BoxOffice_writers

final_df.to_pickle("final_df.pkl")
final_df.head()

,Actors,BoxOffice,Country,Director,Genre,Language,Plot,Production,Rated,Released,...,imdbGrade,tomatoGrade,metacriticGrade,Wins,Nominations,Budget,Awards,BoxOffice_actors,BoxOffice_directors,BoxOffice_writers
0,"{Stephen Lang, Sam Worthington, Sigourney Weav...",749700000,"{Foreign, USA}",{James Cameron},"{Adventure, Fantasy, Sci-Fi, Action}","{Spanish, English}","{home, following, paraplegic, missioncomesrntw...",{20th Century Fox},{PG-13},2009-12-18,...,78,82,83,88,128,237000000,1,8749225092,947816802,1173684544
1,"{Keira Knightley, Geoffrey Rush, Orlando Bloom...",309404152,{USA},{Gore Verbinski},"{Adventure, Fantasy, Action}",{English},"{Sparrownd, Barbossa, find, last, treacheryndt...",{Buena Vista},{PG-13},2007-05-25,...,71,44,50,20,46,300000000,1,11629554832,1329316576,6439780219
2,"{Christoph Waltz, Ralph Fiennes, Daniel Craig,...",208777731,"{Foreign, USA}",{Sam Mendes},"{Adventure, Action, Thriller}","{English, Spanish, Other}","{007, events, missions, organization, sends, c...",{Sony},{PG-13},2015-11-06,...,68,64,60,8,32,245000000,3,6479178314,706988158,4338225868
3,"{Joseph Gordon-Levitt, Christian Bale, Gary Ol...",448130642,"{Foreign, USA}",{Christopher Nolan},"{Action, Thriller}","{Foreign, English}","{Joker, terrorist, Gotham, City, Eight, exile,...",{Warner Bros},{PG-13},2012-07-20,...,84,87,78,38,103,250000000,1,10153049812,1781097215,6820715653
4,"{Samantha Morton, Willem Dafoe, Taylor Kitsch,...",73058679,{USA},{Andrew Stanton},"{Adventure, Sci-Fi, Action}",{English},"{vet, princess, War, creatures, barbarians, Tr...",{Disney},{PG-13},2012-03-09,...,66,51,51,2,8,260000000,0,2742531870,677337921,2082778011


### Data cleaning results

We get a dataframe with the following features:

- Actors, Director, Writer
- Genre
- Budget
- Production
- Plot
- Country, Language
- Release date
- Runtime
- Rated category (PG-13, 18...)

Each movie has one identifyier:

- Title

And we have several labels that define the success of the movie:

- Box Office
- imdb Grade (+ imdb votes)
- Rotten Tomatoes Grade
- Metacritic Grade
- Wins and Nominations

# 3. ML

# 4. Graph creation

After data cleaning, we proceed to the creation of the graph by computing its adjacency matrix. We wish to create an adjacency matrix containing all features. We will hence compute several adjacency matrices that we will then add (using weighting factors). Such matrices are:

- Crew (actors, writers and directors): how many crew members the movies have in common
- Budget: the similarity between movies will be based on how close the budgets are (in a log scale)
- Genre: how many genres two movies have in common
- Production: is it the same production company?
- Plot: how many common words two films have in common
- Language: since the only two categories are English and Foreign, a score between 0 and 2 between 2 movies
- Country: how many countries in common 
- Runtime: the similarity between movies will be based on how close the runtimes are (in a log scale)
- Rated category: binary score = same category or different
- Release: the dissimilarity is how far apart the two movies are in time

In [211]:
final_df = pd.read_pickle("final_df.pkl")
final_df.head()

,Actors,BoxOffice,Country,Director,Genre,Language,Plot,Production,Rated,Released,...,imdbGrade,tomatoGrade,metacriticGrade,Wins,Nominations,Budget,Awards,BoxOffice_actors,BoxOffice_directors,BoxOffice_writers
0,"{Stephen Lang, Sam Worthington, Sigourney Weav...",749700000,"{Foreign, USA}",{James Cameron},"{Adventure, Fantasy, Sci-Fi, Action}","{Spanish, English}","{home, following, paraplegic, missioncomesrntw...",{20th Century Fox},{PG-13},2009-12-18,...,78,82,83,88,128,237000000,0,8749225092,947816802,1173684544
1,"{Keira Knightley, Geoffrey Rush, Orlando Bloom...",309404152,{USA},{Gore Verbinski},"{Adventure, Fantasy, Action}",{English},"{Sparrownd, Barbossa, find, last, treacheryndt...",{Buena Vista},{PG-13},2007-05-25,...,71,44,50,20,46,300000000,0,11629554832,1329316576,6439780219
2,"{Christoph Waltz, Ralph Fiennes, Daniel Craig,...",208777731,"{Foreign, USA}",{Sam Mendes},"{Adventure, Action, Thriller}","{English, Spanish, Other}","{007, events, missions, organization, sends, c...",{Sony},{PG-13},2015-11-06,...,68,64,60,8,32,245000000,0,6479178314,706988158,4338225868
3,"{Joseph Gordon-Levitt, Christian Bale, Gary Ol...",448130642,"{Foreign, USA}",{Christopher Nolan},"{Action, Thriller}","{Foreign, English}","{Joker, terrorist, Gotham, City, Eight, exile,...",{Warner Bros},{PG-13},2012-07-20,...,84,87,78,38,103,250000000,0,10153049812,1781097215,6820715653
4,"{Samantha Morton, Willem Dafoe, Taylor Kitsch,...",73058679,{USA},{Andrew Stanton},"{Adventure, Sci-Fi, Action}",{English},"{vet, princess, War, creatures, barbarians, Tr...",{Disney},{PG-13},2012-03-09,...,66,51,51,2,8,260000000,0,2742531870,677337921,2082778011


## A. Crew adjacency

In [ ]:
weights_crew = np.zeros((len(final_df), len(final_df)))
weights_dir = np.zeros((len(final_df), len(final_df)))
weights_actors = np.zeros((len(final_df), len(final_df)))
weights_writer = np.zeros((len(final_df), len(final_df)))

In [ ]:
for i in range(0, len(final_df)):
    for j in range(i+1, len(final_df)):
        weights_dir[i][j] = weights_dir[i][j] + len(final_df['Director'][i] & final_df['Director'][j])
weights_dir = weights_dir + weights_dir.T

for i in range(0, len(final_df)):
    for j in range(i+1, len(final_df)):
        weights_actors[i][j] = weights_actors[i][j] + len(final_df['Actors'][i] & final_df['Actors'][j])
weights_actors = weights_actors + weights_actors.T

for i in range(0, len(final_df)):
    for j in range(i+1, len(final_df)):
        weights_writer[i][j] = weights_writer[i][j] + len(final_df['Writer'][i] & final_df['Writer'][j])
weights_writer = weights_writer + weights_writer.T

In [ ]:
# Add and save
weights_crew = weights_dir + weights_actors + weights_writer

np.save('weights_dir.npy', weights_dir)
np.save('weights_act.npy', weights_actors)
np.save('weights_wri.npy', weights_writer)
np.save('weights_crew.npy', weights_crew)

## B. Budget adjacency
### TODO: check the computing method

Two similar films are two films that have a similar budget. This is why we cannot use the absolute difference between the films as a similarity metric. 

Moreover, two films having a budget of millions but being 1 million dollars appart are more similar than a film having a budget of 10 dollars and a film having a budget of 1 million and 10 dollars. This is why before computing the absolute difference in budget, we must convert the budget to a logarithmic scale. 

Then, once the differences have been computed, the weight matrix has to be:
$ weights = max(weights) - weights$

In [ ]:
weights_budget = np.zeros((len(final_df), len(final_df)))

for i in range(0, len(final_df)):
    for j in range(i+1, len(final_df)):
        weights_budget[i][j] = np.abs(np.log10(final_df['Budget'][i]) -  np.log10(final_df['Budget'][j]))

# Symmetrize
weights_budget = weights_budget + weights_budget.T
weights_budget = np.max(weights_budget) - weights_budget

# Save 
np.save('weights_budget.npy', weights_budget)

## C. Genre adjacency

In [ ]:
weights_genre = np.zeros((len(final_df), len(final_df)))

for i in range(0, len(final_df)):
    for j in range(i+1, len(final_df)):
        weights_genre[i][j] = len(final_df['Genre'][i] & final_df['Genre'][j])

# Symmetrize
weights_genre = weights_genre + weights_genre.T

# Save 
np.save('weights_genre.npy', weights_genre)

## D. Production 

In [ ]:
weights_prod = np.zeros((len(final_df), len(final_df)))

for i in range(0, len(final_df)):
    for j in range(i+1, len(final_df)):
        weights_prod[i][j] = len(final_df['Production'][i] & final_df['Production'][j])

# Symmetrize
weights_prod = weights_prod + weights_prod.T

# Save 
np.save('weights_prod.npy', weights_prod)

## E. Plot

In [ ]:
weights_plot = np.zeros((len(final_df), len(final_df)))

for i in range(0, len(final_df)):
    for j in range(i+1, len(final_df)):
        weights_plot[i][j] = len(final_df['Plot'][i] & final_df['Plot'][j])

# Symmetrize
weights_plot = weights_plot + weights_plot.T

# Save 
np.save('weights_plot.npy', weights_plot)

## F. Country, Language

In [ ]:
weights_lang = np.zeros((len(final_df), len(final_df)))

for i in range(0, len(final_df)):
    for j in range(i+1, len(final_df)):
        weights_lang[i][j] = len(final_df['Language'][i] & final_df['Language'][j])

for i in range(0, len(final_df)):
    for j in range(i+1, len(final_df)):
        weights_lang[i][j] = weights_lang[i][j] + len(final_df['Country'][i] & final_df['Country'][j])

# Symmetrize        
weights_lang = weights_lang + weights_lang.T

# Save
np.save('weights_lang.npy', weights_lang)

## G. Release date
### TODO: check computing method

In [ ]:
weights_date = np.zeros((len(final_df), len(final_df)))

for i in range(0, len(final_df)):
    for j in range(i+1, len(final_df)):
        if (not pd.isnull(final_df['Released'][i])) and (not pd.isnull(final_df['Released'][j])):
            weights_date[i][j] = abs(final_df['Released'][i] - final_df['Released'][j]).days
        else:
            weights_date[i][j] = 37742

# Symmetrize        
weights_date = weights_date + weights_date.T

# Save
np.save('weights_date.npy', weights_date)

## H. Runtime
### TODO: check computing method

In [ ]:
weights_runtime = np.zeros((len(final_df), len(final_df)))

for i in range(0, len(final_df)):
    for j in range(i+1, len(final_df)):
        weights_runtime[i][j] = np.abs(final_df['Runtime'][i] -  final_df['Runtime'][j])

# Symmetrize
weights_runtime = weights_runtime + weights_runtime.T
weights_runtime = np.max(weights_runtime) - weights_runtime

# Save 
np.save('weights_runtime.npy', weights_runtime)

## I. Rated category

In [ ]:
weights_rated = np.zeros((len(final_df), len(final_df)))

for i in range(0, len(final_df)):
    for j in range(i+1, len(final_df)):
        weights_rated[i][j] = len(final_df['Rated'][i] & final_df['Rated'][j])

# Symmetrize
weights_rated = weights_rated + weights_rated.T

# Save 
np.save('weights_rated.npy', weights_rated)

In [ ]:
weights_crew = np.load('weights_crew.npy')
weights_budget = np.load('weights_budget.npy')
weights_genre = np.load('weights_genre.npy')
weights_prod = np.load('weights_prod.npy')
weights_plot = np.load('weights_plot.npy')
weights_lang = np.load('weights_lang.npy')
weights_date = np.load('weights_date.npy')
weights_runtime = np.load('weights_runtime.npy')
weights_rated = np.load('weights_rated.npy')

## Total adjacency matrix

Now, we need to add all weight matrices. It is however of use to add them using weigh factors, as some features may be more important than others.

In [ ]:
crew_factor = 1
budget_factor = 1
genre_factor = 1
prod_factor = 1
plot_factor = 1
lang_factor = 1
date_factor = 1
runtime_factor = 1
rated_factor = 1
sum_factors = (crew_factor + budget_factor + genre_factor + prod_factor + 
               plot_factor + lang_factor + date_factor + runtime_factor + rated_factor)

weights = (crew_factor*weights_crew + budget_factor*weights_budget + genre_factor*weights_genre + 
           prod_factor*weights_prod + plot_factor*weights_plot + lang_factor*weights_lang + 
           date_factor*weights_date + runtime_factor*weights_runtime + rated_factor*weights_rated) / sum_factors

# The adjacency matrix is the normalized weight matrix
adjacency = weights / np.max(weights)

# Save
np.save('weights.npy', weights)
np.save('adjacency.npy', adjacency)

In [ ]:
weights = np.load('weights.npy')
adjacency = np.load('adjacency.npy')